In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from model import lorenz63_fdm, M63
from assimilation import OI, ExtendedKF, M3DVar, EnKF, Hybrid3DEnVar, OI

In [2]:
X_nature = np.load('./data/X_nature.npy')
X_ini = np.load('./data/X_ini.npy')
ts = np.load('./data/time_span.npy')
Pb = np.load('./data/Pb.npy')
R = np.load('./data/R.npy')

dt = 0.01

# generate initial ensemble
N_ens = 30
rng = np.random.RandomState(42)
X_ens_ini = rng.multivariate_normal(X_ini.ravel(), Pb, size=N_ens).T  # (3, N_ens)

In [3]:
obs_intv = 8

enkf_params = {
    'X_ens_ini': X_ens_ini, 
    'obs_interv': obs_intv, 
    'alpha': 0.3,
    'inflat': 1.4
}

ekf_params = {
    'X_ini': X_ini,
    'obs_interv': obs_intv,
    'Pb': Pb,
    'M': lambda X: M63(X.ravel(), dt),
    'alpha': 0.3,
    'inflat': 1.4
}

tdv_params = {
    'X_ini': X_ini,
    'obs_interv': obs_intv,
    'Pb': Pb,
}

hy3denvar_params = {
    'X_ini': X_ini,
    'X_ens_ini': X_ens_ini,
    'obs_interv': obs_intv,
    'Pb': Pb,
    'alpha': 0.2,
    'inflat': 1.6,
    'beta': 0.5
}

da_params = {
    'enkf': enkf_params,
    'ekf': ekf_params,
    '3dvar': tdv_params,
    '3denvar': hy3denvar_params
}

# Skewness: alpha=1.30~3.30

In [4]:
from scipy.stats import skewnorm

def gen_skewnormal(mean, var, alpha, size, random_state=None):
    """generate random number by skewnorm, and adjust them into given mean and variance"""
    # generate standard skew normal distribution
    X = skewnorm.rvs(alpha, loc=0, scale=1, size=size, random_state=random_state)
    
    # theory expectation value (mean) and variance of standard skew normal distribution
    tmean = np.sqrt(2/np.pi) * alpha / np.sqrt(1+alpha**2)
    tvar = 1 - 2/np.pi * alpha**2 / (1+alpha**2)

    # adjust var, then adjust mean
    X = np.sqrt(var/tvar) * X
    tmean = np.sqrt(var/tvar) * np.sqrt(2/np.pi) * alpha / np.sqrt(1+alpha**2)
    X = X + mean - tmean
    
    return X

def gen_skewobs(nature, mean, var, alpha, obs_intv, random_state=None):
    """generate skew observations"""
    ndim, size = nature.shape
    
    if isinstance(mean, (int, float)):
        mean = [mean for _ in range(ndim)]
    if isinstance(var, (int, float)):
        var = [var for _ in range(ndim)]
    if isinstance(alpha, (int, float)):
        alpha = [alpha for _ in range(ndim)]
        
    if random_state is None:
        random_state = [np.random.randint(0, 100) for _ in range(ndim)]
    
    obs = np.zeros((ndim, size//obs_intv))
    for irow, (m, v, a, rst) in enumerate(zip(mean, var, alpha, random_state)):
        samples = gen_skewnormal(m, v, a, size, rst)
        obs[irow,:] = (nature[irow,:] + samples)[::obs_intv]
        
    return obs

In [12]:
def assim_exp(var):
    alphas = [1.30, 1.55, 1.80, 2.05, 2.30, 2.55, 2.80, 3.05, 3.30]

    da_schemes = ['enkf', 'ekf', '3dvar', '3denvar']
    da_objs = [EnKF, ExtendedKF, M3DVar, Hybrid3DEnVar]

    repeat_times = 10
    for t in range(repeat_times):
        print(t)
        random_states = np.random.randint(0, 100, size=(len(alphas), 3))

        for da_obj, da_str in zip(da_objs, da_schemes):
            
            ###### DO NOT test EKF because tech difficault #####
            if da_str == 'ekf' or da_str == '3denvar':
                continue
            ####################################################
            
            for alpha, rsts in zip(alphas, random_states):
                print(da_str, alpha)

                obs = gen_skewobs(X_nature, 0, 2, alpha, obs_intv, rsts)
                if var == 'X':
                    obs = obs[[0],:]
                    R = np.array([[2]])
                    H_func = lambda arr: arr[[0]]
                    H = np.array([[1, 0, 0]])
                elif var == 'Y':
                    obs = obs[[1],:]
                    R = np.array([[2]])
                    H_func = lambda arr: arr[[1]]
                    H = np.array([[0, 1, 0]])
                elif var == 'Z':
                    obs = obs[[2],:]
                    R = np.array([[2]])
                    H_func = lambda arr: arr[[2]]
                    H = np.array([[0, 0, 1]])
                    
                da = da_obj(lorenz63_fdm, dt)
                
                params = da_params[da_str]
                params['obs'] = obs
                params['R'] = R
                params['H_func'] = H_func
                if 'H' in da.list_params():
                    params['H'] = H
                    
                da.set_params(**params)
                da.cycle()

                path = './repeat_experiments_partial/' + var
                fn = f'{path}/{da_str}_analysis_alpha{alpha*100:03.0f}_t{t}'
                if da_str != '3denvar':
                    np.save(fn, da.analysis)
                else:
                    np.save(fn, da.analysis_3dvar)

                fn = f'{path}/obs_alpha{alpha*100:03.0f}_t{t}'
                np.save(fn, obs)    

In [13]:
# assimilate x only
assim_exp('X')

0
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.3
3dvar 1.55
3dvar 1.8
3dvar 2.05
3dvar 2.3
3dvar 2.55
3dvar 2.8
3dvar 3.05
3dvar 3.3
1
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.3
3dvar 1.55
3dvar 1.8
3dvar 2.05
3dvar 2.3
3dvar 2.55
3dvar 2.8
3dvar 3.05
3dvar 3.3
2
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.3
3dvar 1.55
3dvar 1.8
3dvar 2.05
3dvar 2.3
3dvar 2.55
3dvar 2.8
3dvar 3.05
3dvar 3.3
3
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.3
3dvar 1.55
3dvar 1.8
3dvar 2.05
3dvar 2.3
3dvar 2.55
3dvar 2.8
3dvar 3.05
3dvar 3.3
4
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.3
3dvar 1.55
3dvar 1.8
3dvar 2.05
3dvar 2.3
3dvar 2.55
3dvar 2.8
3dvar 3.05
3dvar 3.3
5
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.

In [14]:
# assimilate y only
assim_exp('Y')

0
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.3
3dvar 1.55
3dvar 1.8
3dvar 2.05
3dvar 2.3
3dvar 2.55
3dvar 2.8
3dvar 3.05
3dvar 3.3
1
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.3
3dvar 1.55
3dvar 1.8
3dvar 2.05
3dvar 2.3
3dvar 2.55
3dvar 2.8
3dvar 3.05
3dvar 3.3
2
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.3
3dvar 1.55
3dvar 1.8
3dvar 2.05
3dvar 2.3
3dvar 2.55
3dvar 2.8
3dvar 3.05
3dvar 3.3
3
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.3
3dvar 1.55
3dvar 1.8
3dvar 2.05
3dvar 2.3
3dvar 2.55
3dvar 2.8
3dvar 3.05
3dvar 3.3
4
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.3
3dvar 1.55
3dvar 1.8
3dvar 2.05
3dvar 2.3
3dvar 2.55
3dvar 2.8
3dvar 3.05
3dvar 3.3
5
enkf 1.3
enkf 1.55
enkf 1.8
enkf 2.05
enkf 2.3
enkf 2.55
enkf 2.8
enkf 3.05
enkf 3.3
3dvar 1.